<a href="https://colab.research.google.com/github/ask350/UM-Projects/blob/main/Animal%20Classification/animalclass_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Animal Classification Streamlit App

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image
import json
import os

# Load model and class labels
model = tf.keras.models.load_model("animal_classifier_model.keras")
with open("class_names.txt", "r") as f:
    class_names = json.load(f)

st.title("🐾 Animal Image Classifier")
st.markdown("Upload an image or select one to classify.")

# Show supported animal classes
st.markdown("Supported Animal Classes:")
st.markdown(", ".join(class_names))

In [ ]:
# Upload or Choose Sample Image
upload_option = st.radio("Choose image source:", ("Upload", "Sample images"))

img = None  # Initialize

if upload_option == "Upload":
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_file:
        img = Image.open(uploaded_file).convert("RGB")
else:
    # Sample image directory (you must add some sample images in a folder)
    sample_dir = "test_images"
    if os.path.exists(sample_dir):
        sample_files = [f for f in os.listdir(sample_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        selected_image = st.selectbox("Choose a sample image", sample_files)
        if selected_image:
            img = Image.open(os.path.join(sample_dir, selected_image)).convert("RGB")
    else:
        st.warning("Sample image folder not found.")


In [ ]:
# Show image preview
if img:
    st.image(img, caption="Selected Image", use_column_width=True)

# Predict button
if st.button("Classify") and img:
    # Preprocess
    img_resized = img.resize((224, 224))
    img_array = image.img_to_array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)

    # Show result
    st.subheader(f"Prediction: **{predicted_class}**")
    st.write(f"Confidence: `{confidence*100:.2f}%`")